In [1]:
# 用于将下载的csv文件合并、转换成h5文件

In [2]:
# 导入包
import pandas as pd
import numpy as np
import h5py
import multiprocessing
import os

In [3]:
# 参数设置
time_start = '201901' #时间格式，必须全为数字
time_end = '201912'
folder_path = "./Download" # 源数据
h5_file_name = 'NYC_Bike.csv' # h5 文件名称
h5_path = './raw_bike_data/' # 目标路径
h5_file = h5_path + h5_file_name
multiprocessing.cpu_count()


16

In [7]:

# 设置文件夹路径
folder_path = "./Download"
# 使用os.walk()函数遍历文件夹中的所有文件
csv_files = []
for root, dirs, files in os.walk(folder_path):
    print('要合并的文件有:')
    for filename in files:
        # 打印文件路径
        year_month = filename[:6]
        if int(year_month) >= int(time_start) and int(year_month) <= int(time_end):
            file_path = os.path.join(root, filename)
            # print(file_path)
            csv_files.append(file_path)
            csv_files = sorted(csv_files)
print(csv_files)

要合并的文件有:
['./Download/201901-citibike-tripdata.csv', './Download/201902-citibike-tripdata.csv', './Download/201903-citibike-tripdata.csv', './Download/201904-citibike-tripdata.csv', './Download/201905-citibike-tripdata.csv', './Download/201906-citibike-tripdata.csv', './Download/201907-citibike-tripdata.csv', './Download/201908-citibike-tripdata.csv', './Download/201909-citibike-tripdata.csv', './Download/201910-citibike-tripdata.csv', './Download/201911-citibike-tripdata.csv', './Download/201912-citibike-tripdata.csv']


In [8]:
# 多进程读取CSV文件

num_processes = multiprocessing.cpu_count()

def preprocess_nyc_bike(csv_file):
    df = pd.read_csv(csv_file)
    drop_columns = ['tripduration','start station id','start station name','end station id','end station name','bikeid','usertype','birth year','gender']
    df.drop(drop_columns,axis=1,inplace=True)
    df.columns = ['stime','etime','slat','slon','elat','elon']
    return df

with multiprocessing.Pool(num_processes) as pool:
    dfs = pool.map(preprocess_nyc_bike, csv_files)

In [9]:
dfs

[                           stime                     etime       slat  \
 0       2019-01-01 00:01:47.4010  2019-01-01 00:07:07.5810  40.778968   
 1       2019-01-01 00:04:43.7360  2019-01-01 00:10:00.6080  40.751873   
 2       2019-01-01 00:06:03.9970  2019-01-01 00:15:55.4380  40.785247   
 3       2019-01-01 00:07:03.5450  2019-01-01 00:52:22.6500  40.732219   
 4       2019-01-01 00:07:35.9450  2019-01-01 00:12:39.5020  40.727434   
 ...                          ...                       ...        ...   
 967282  2019-01-31 17:33:26.4490  2019-01-31 17:38:17.8940  40.668132   
 967283  2019-01-31 18:57:45.3380  2019-01-31 19:05:02.4970  40.668132   
 967284  2019-01-31 19:11:41.1930  2019-01-31 19:14:34.3350  40.668132   
 967285  2019-01-31 20:54:51.1440  2019-01-31 20:58:55.3100  40.668132   
 967286  2019-01-31 23:38:42.1770  2019-01-31 23:45:09.4520  40.668132   
 
              slon       elat       elon  
 0      -73.973747  40.788221 -73.970416  
 1      -73.977706  40.7

In [10]:
# 合并dataframe并写入h5文件
df = pd.concat(dfs)
df.to_csv('raw_bike_data/NYC_Bike.csv',index=False)
df

,stime,etime,slat,slon,elat,elon
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,40.778968,-73.973747,40.788221,-73.970416
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,40.751873,-73.977706,40.747804,-73.973442
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,40.785247,-73.976673,40.773142,-73.958562
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,40.732219,-73.981656,40.738046,-73.996430
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,40.727434,-73.993790,40.738274,-73.987520
...,...,...,...,...,...,...
955205,2019-12-31 23:58:18.0160,2019-12-31 23:59:51.6360,40.765005,-73.958185,40.761227,-73.960940
955206,2019-12-31 23:58:59.8960,2020-01-01 00:12:06.3460,40.751551,-73.993934,40.768254,-73.988639
955207,2019-12-31 23:59:03.6950,2020-01-01 00:04:54.8730,40.672968,-73.970880,40.672815,-73.983524
955208,2019-12-31 23:59:21.3610,2020-01-01 00:25:32.9420,40.732264,-73.998522,40.693261,-73.968896
